In [ ]:
!pip3 install datasets
from datasets import load_dataset

dataset = load_dataset("interpress_news_category_tr_lite")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
import os
import sys
import csv
import warnings
import re
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn import metrics
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import os

!pip install ipython-autotime
%load_ext autotime

# Error: field larger than field limit (131072)
csv.field_size_limit(sys.maxsize)
warnings.filterwarnings('ignore')

pd.set_option("max_colwidth", 400)

#tf.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00
time: 1.07 ms (started: 2024-01-16 08:47:34 +00:00)


In [ ]:
train_dataset = load_dataset("interpress_news_category_tr_lite", split="train")
test_dataset = load_dataset("interpress_news_category_tr_lite", split="test")

df_train = train_dataset.to_pandas()
df_test = test_dataset.to_pandas()
data = pd.concat([df_train, df_test], ignore_index=True)

data.drop_duplicates(subset='content', keep='first', inplace=True, ignore_index=True)

sil_satinlar = [2, 3, 4, 5, 7, 9]
data = data[~data['category'].isin(sil_satinlar)]
data['category'].replace({0: 0, 1: 1, 6: 2, 8: 3}, inplace=True)

data["words"] = [len(x.split()) for x in data["content"].tolist()]

min_content_size = 50
max_content_size = 500

data = data[data['words']>=min_content_size]
data = data[data['words']<=max_content_size]

time: 20.5 s (started: 2024-01-16 08:52:33 +00:00)


In [ ]:
len(data)

13667

time: 4.8 ms (started: 2024-01-16 07:50:57 +00:00)


In [ ]:
def create_subset(data, ratio=0.05):
    subset_size = int(len(data) * ratio)
    return data.sample(subset_size, random_state=42)

data = create_subset(data, ratio=0.2)

time: 10.3 ms (started: 2024-01-16 08:52:53 +00:00)


In [ ]:
data = data.sample(frac=1)

features, targets = data['content'], data['category']

train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.8,
        test_size=0.2,
        random_state=42,
        shuffle=True,
        stratify=targets
    )


print("Train Data Set size: ", len(train_features))
print("Test Data Set size: ", len(test_features))

Train Data Set size:  21868
Test Data Set size:  5467
time: 22.6 ms (started: 2024-01-16 08:52:53 +00:00)


In [ ]:
train_text_ds_raw = tf.data.Dataset.from_tensor_slices(tf.cast(train_features.values, tf.string))
train_cat_ds_raw = tf.data.Dataset.from_tensor_slices(tf.cast(train_targets.values, tf.int64),)
vocab_size = 50000
max_len = max_content_size

@tf.keras.utils.register_keras_serializable()
def custom_standardization(input_string):
    no_uppercased = tf.strings.lower(input_string, encoding='utf-8')
    no_stars = tf.strings.regex_replace(no_uppercased, "\*", " ")
    no_html = tf.strings.regex_replace(no_stars, "<br />", "")
    no_digits = tf.strings.regex_replace(no_html, "\w*\d\w*","")
    no_punctuations = tf.strings.regex_replace(no_digits, f"([{string.punctuation}])", r" ")
    no_extra_space = tf.strings.regex_replace(no_punctuations, " +"," ")
    #remove Turkish chars
    no_I = tf.strings.regex_replace(no_extra_space, "ı","i")
    no_O = tf.strings.regex_replace(no_I, "ö","o")
    no_C = tf.strings.regex_replace(no_O, "ç","c")
    no_S = tf.strings.regex_replace(no_C, "ş","s")
    no_G = tf.strings.regex_replace(no_S, "ğ","g")
    no_U = tf.strings.regex_replace(no_G, "ü","u")
    return no_U


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=max_len,
)

vectorize_layer.adapt(train_text_ds_raw)

In [ ]:
vocab = vectorize_layer.get_vocabulary()

vectorizer_model = tf.keras.models.Sequential()
vectorizer_model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
vectorizer_model.add(vectorize_layer)

def prepare_lm_inputs_labels(text):
    text = tf.expand_dims(text, -1)
    return tf.squeeze(vectorize_layer(text))
    
train_text_ds = train_text_ds_raw.map(prepare_lm_inputs_labels,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_ds = tf.data.Dataset.zip(
    (       train_text_ds,
            train_cat_ds_raw
        )
)

train_size = train_ds.cardinality().numpy()

In [ ]:
test_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(test_features.values, tf.string)
)

test_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(test_targets.values, tf.int64),

)

test_text_ds = test_text_ds_raw.map(prepare_lm_inputs_labels,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_ds = tf.data.Dataset.zip(
    (       test_text_ds,
            test_cat_ds_raw
       )
)

test_size = test_ds.cardinality().numpy()

In [ ]:
batch_size=64
AUTOTUNE=tf.data.experimental.AUTOTUNE

train_ds=train_ds.shuffle(buffer_size=train_size)
train_ds=train_ds.batch(batch_size=batch_size,drop_remainder=True)
train_ds=train_ds.cache()
train_ds = train_ds.prefetch(AUTOTUNE)

test_ds=test_ds.shuffle(buffer_size=train_size)
test_ds=test_ds.batch(batch_size=batch_size,drop_remainder=True)
test_ds=test_ds.cache()
test_ds = test_ds.prefetch(AUTOTUNE)

In [ ]:
desired_time_steps = 64
desired_feature_dim = 500

def reshape_data(x, y):
    x_reshaped = tf.reshape(x, (desired_time_steps, desired_feature_dim))
    return x_reshaped, y

train_ds = train_ds.map(reshape_data)
test_ds = test_ds.map(reshape_data)

In [ ]:
# FNN

embed_dim = 4
feed_forward_dim = 8
def create_model_FFN():
    inputs_tokens = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding_layer = layers.Embedding(input_dim=vocab_size,
                                       output_dim=embed_dim,
                                       input_length=max_len)
    x = embedding_layer(inputs_tokens)
    x = layers.Flatten()(x)
    dense_layer = layers.Dense(feed_forward_dim, activation='relu')
    x = dense_layer(x)
    x = layers.Dropout(.5)(x)
    outputs = layers.Dense(4)(x)

    model = keras.Model(inputs=inputs_tokens, outputs=outputs, name='model_FFN')
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer="adam", loss=loss_fn, metrics=metric_fn)
    return model

model_FFN = create_model_FFN()

model_FFN.summary()

In [ ]:
history = model_FFN.fit(train_ds, verbose=2, epochs=25)

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('Feed Forward Network Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model_FFN loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model_FFN.evaluate(test_ds)
print("Test Dataset Accuracy: ", accuracy)

In [ ]:
preds = model_FFN.predict(test_ds)
preds = preds.argmax(axis=1)
actuals = test_ds.unbatch().map(lambda x,y: y)
actuals=list(actuals.as_numpy_iterator())

from sklearn import metrics
print(metrics.classification_report(actuals, preds, digits=4))

In [ ]:
# CONV1D

embed_dim = 4
feed_forward_dim = 8

def create_model_Conv1D():
    inputs_tokens = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding_layer = layers.Embedding(input_dim=vocab_size,
                                       output_dim=embed_dim,
                                       input_length=max_len)
    x = embedding_layer(inputs_tokens)
    x = layers.Conv1D(filters=256, kernel_size=7,
                      padding='same', activation='relu')(x)
    x = layers.Conv1D(filters=128, kernel_size=5,
                      padding='same', activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(feed_forward_dim, activation='sigmoid')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(4)(x)
    model = keras.Model(inputs=inputs_tokens,
                        outputs=outputs, name='model_Conv1D')
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn  = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer="adam", loss=loss_fn, metrics=metric_fn)

    return model

model_Conv1D = create_model_Conv1D()
model_Conv1D.summary()

In [ ]:
history = model_Conv1D.fit(train_ds, verbose=1, epochs=15)

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('Conv1D Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model_Conv1D loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model_Conv1D.evaluate(test_ds)
print("Test Accuracy: ", accuracy)

In [ ]:
preds = model_Conv1D.predict(test_ds)
preds = preds.argmax(axis=1)
actuals = test_ds.unbatch().map(lambda x, y:y)
actuals = list(actuals.as_numpy_iterator())

In [ ]:
print(metrics.classification_report(actuals, preds, digits=4))

In [ ]:
# LSTM

embed_dim = 4
feed_forward_dim = 8

def create_model_LSTM():
    inputs_tokens = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding_layer = layers.Embedding(input_dim=vocab_size,
                                       output_dim=embed_dim,
                                       input_length=max_len)
    x = embedding_layer(inputs_tokens)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(feed_forward_dim, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    outputs = layers.Dense(4)(x)
    model = keras.Model(inputs=inputs_tokens,
                        outputs=outputs, name='model_LSTM')
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn  = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer="adam", loss=loss_fn, metrics=metric_fn)

    return model

model_LSTM=create_model_LSTM()
model_LSTM.summary()

In [ ]:
history = model_LSTM.fit(train_ds, verbose=1, epochs=5)

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('model_LSTM Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model_LSTM loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model_LSTM.evaluate(test_ds)
print("Test accuracy: ", accuracy)

In [ ]:
preds = model_LSTM.predict(test_ds)
preds = preds.argmax(axis=1)
actuals = test_ds.unbatch().map(lambda x, y:y)
actuals = list(actuals.as_numpy_iterator())

In [ ]:
print(metrics.classification_report(actuals, preds, digits=4))

In [ ]:
# BERT

!pip install simpletransformers
!pip install tensorflow==2.14
import tensorflow as tf
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.max_seq_length=500
model_args.reprocess_input_data= True
model_args.overwrite_output_dir= True
model_args.train_batch_size= 32
model_args.fp16=False
model_args.output_dir= "bert_model"
model_args.learning_rate = 1e-4
model_args.no_save= True
model_args.save_eval_checkpoints=False
model_args.save_model_every_epoch=False
model_args.save_optimizer_and_scheduler=False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.is_available()

In [ ]:
tf.test.gpu_device_name()

In [ ]:
#create model with turkish bert
model_bert = ClassificationModel('bert',
                                 'dbmdz/bert-base-turkish-uncased',
                                 num_labels=4,
                                  args=model_args)

def custom_standardization(input_string):
    no_uppercased = tf.strings.lower(input_string, encoding='utf-8')
    no_stars = tf.strings.regex_replace(no_uppercased, "\*", " ")
    no_repeats = tf.strings.regex_replace(no_stars, "devamını oku", "")
    no_html = tf.strings.regex_replace(no_repeats, "<br />", "")
    no_digits = tf.strings.regex_replace(no_html, "\w*\d\w*","")
    no_punctuations = tf.strings.regex_replace(no_digits, f"([{string.punctuation}])", r" ")
    no_extra_space = tf.strings.regex_replace(no_punctuations, " +"," ")

    no_I = tf.strings.regex_replace(no_extra_space, "ı","i")
    no_O = tf.strings.regex_replace(no_I, "ö","o")
    no_C = tf.strings.regex_replace(no_O, "ç","c")
    no_S = tf.strings.regex_replace(no_C, "ş","s")
    no_G = tf.strings.regex_replace(no_S, "ğ","g")
    no_U = tf.strings.regex_replace(no_G, "ü","u")

    return no_U.numpy().decode("utf-8")

train_txt=train_features.apply(custom_standardization)
test_txt=test_features.apply(custom_standardization)

train=pd.concat([train_txt, train_targets], axis=1)
test=pd.concat([test_txt, test_targets], axis=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_bert.train_model(train)

  0%|          | 0/21868 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/2734 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/2734 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_predictions = model_bert.eval_model(test)

  0%|          | 0/1367 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/171 [00:00<?, ?it/s]

time: 51.8 s (started: 2024-01-16 08:41:09 +00:00)


In [ ]:
result

{'mcc': 0.8679144019802999, 'eval_loss': 0.5224905760454325}

time: 3.31 ms (started: 2024-01-16 08:42:08 +00:00)


In [ ]:
predictions = model_outputs.argmax(axis=1)
actuals = test_targets.values

time: 703 µs (started: 2024-01-16 08:42:11 +00:00)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.9092904169714704

time: 4.16 ms (started: 2024-01-16 08:42:15 +00:00)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.931     0.919     0.925       618
           1      0.895     0.918     0.906       342
           2      0.903     0.903     0.903       259
           3      0.864     0.858     0.861       148

    accuracy                          0.909      1367
   macro avg      0.898     0.900     0.899      1367
weighted avg      0.909     0.909     0.909      1367

time: 24.4 ms (started: 2024-01-16 08:42:26 +00:00)


In [ ]:
model_bert.save_model("drive/MyDrive/bert_model")

time: 1.76 ms (started: 2024-01-16 08:42:47 +00:00)


In [ ]:
from google.colab import drive

drive.mount('/gdrive')

Mounted at /gdrive
time: 8.49 s (started: 2024-01-16 08:44:27 +00:00)


In [ ]:
model_bert.save_model('bert_base_turkish_uncase_text_classification')

time: 17.1 ms (started: 2024-01-16 08:45:37 +00:00)


In [ ]:
!mv bert_model /gdrive/

In [ ]:
model_bert = ClassificationModel('bert', '/content/drive/MyDrive/bert_model', num_labels=4, args=model_args)